In [24]:
# Import the Twython class
import tweepy
import csv
import time
import json
import datetime
import pandas as pd
import emoji
import os
from json import dumps
import preprocessor as p

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [36]:
#Mysql
import pymysql
import json
from functools import partial
import csv
import jsonpickle

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)
            
def return_db_results(connection, query, args=None):#Gambiarra para pegar o id
    with connection.cursor() as cursor:
        cursor.execute(query, args)
        for result in cursor:
            return result #retorna logo o primeiro
connection = pymysql.connect(
    host='localhost',
    user='root',
    password='vlm1998',
    database='redes')

db = partial(run_db_query, connection)


ModuleNotFoundError: No module named 'unicode'

In [26]:
#Função do DB para retornar as tags dos jogadores
def return_players_tag(connection,id_jogador):    

    with connection.cursor() as cursor:

        cursor.execute('SELECT tag FROM jogador_tag WHERE id_jogador = %s', (id_jogador) ) ## irá retornar a tag do jogador no twitter

        res = cursor.fetchall()

        tags = tuple(x[0] for x in res)

        return tags

In [27]:
db('SELECT * FROM jogador_tag')


Executando query:
(1, 'alaatweets')
(3, 'kaj33')
(6, 'ShareefAbdur_R')
(10, 'alexabrines')
(15, 'QuincyAcy')
(19, '10jadams')
(20, 'jordanadams1231')
(22, 'RealStevenAdams')
(24, 'Bam1of1')
(27, 'Adrien4')
(29, 'moeager')
(32, 'danielrainge')
(33, 'AjincaAlexis42')
(40, 'furkanaldemir19')
(41, 'colea45')
(42, 'aldridge_12')
(44, 'CAlexander')
(52, 'Iam_RawleAlkins')
(54, 'GraysonJAllen')
(55, '_bigjayy_')
(60, 'THEleekAllen30')
(62, 'greenRAYn20')
(63, 'aa000G9')
(67, 'FreeMoAlmond')
(71, 'JohnAmaechi')
(73, 'farouq1')
(74, 'LouAmundson17')
(78, 'ASqared06')
(88, 'JusAnderson1')
(89, 'chibbs_1')
(91, 'KyleAnderson5')
(103, 'IkePostrs_')
(106, 'Giannis_An34')
(107, 'Kostas_ante13')
(109, 'carmeloanthony')
(110, 'GregAnthony50')
(113, 'OAnunoby')
(118, 'RobertArchibald')
(119, 'RyArch15')
(121, 'agentzeroshow')
(122, 'TrevorAriza')
(125, 'bjarmstrong')
(136, 'DarrellArthur00')
(138, 'melartis1')
(139, 'AsikOmer')
(157, 'Gustavo_Ayon15')
(159, 'DeandreAyton')
(160, 'KAzubuike7')
(162, 'Re

In [31]:
tags = return_players_tag(connection,41)
print(tags[0])
user_tweet = tags[0]
print(user_tweet)

colea45
colea45


In [32]:
try:
    os.remove("saved_tweets.json")
except:
    pass

In [37]:

class TweetMiner(object):

    result_limit    =   20    
    data            =   []
    api             =   False
    
    with open("twitter_credentials.json", "r") as file:
        creds = json.load(file)

    
    
    def __init__(self, keys_dict=creds, api=api,
                 result_limit = 20):
        
        
        auth = tweepy.OAuthHandler(
            creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
        auth.set_access_token(
            creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
        
        api = tweepy.API(auth)
        max_id = -1
        sinceId = None
        #Date that starts after NBA finals until the beggining of the season
        startDate = datetime.datetime(2019, 6, 30, 0, 0, 0)
        endDate = datetime.datetime(2019, 10, 22, 0, 0, 0)
        
        #however many you want to limit your collection to.  how much storage space do you have?
        maxTweets = 10000000 
        dict_json = {}
        tweetCount = 0
        searchQuery = '@JimmyButler' 
        searchQuery += '-filter:retweets'
        with open("saved_tweets.json", "wt") as f:
            while tweetCount < maxTweets:
                    try:
                        if (max_id <= 0):
                            if (not sinceId):
                                new_tweets = api.search(q=searchQuery, count=100)
                            else:
                                new_tweets = api.search(q=searchQuery, count=100,
                                                        since_id=sinceId)
                        else:
                            if (not sinceId):
                                new_tweets = api.search(q=searchQuery, count=100,
                                                        max_id=str(max_id - 1))
                            else:
                                new_tweets = api.search(q=searchQuery, count=100,
                                                        max_id=str(max_id - 1),
                                                        since_id=sinceId)
                        if not new_tweets:
                            print("No more tweets found")
                            break
                            
                        for tweet in new_tweets:
                            print(tweet.created_at)
                            #if(tweet.created_at < endDate and tweet.created_at > startDate):
                            if(searchQuery in dict_json):
                                l_values = []
                                l_values.append(tweet.text)
                                l_values.append(str(tweet.created_at))
                                dict_json[searchQuery].append(l_values)
                            else:
                                l_values = []
                                l_values.append(tweet.text)
                                l_values.append(str(tweet.created_at))
                                dict_json[searchQuery] = l_values
                                
                            
                        tweetCount += len(new_tweets)
                        print("Downloaded {0} tweets".format(tweetCount))
                        max_id = new_tweets[-1].id
                    except tweepy.TweepError as e:
                        # Just exit if any error
                        print("some error : " + str(e))
                        break
            json.dump(dict_json, f, indent = 10)

miner = TweetMiner()

2019-11-08 19:57:59
2019-11-08 19:52:21
2019-11-08 19:50:35
2019-11-08 19:47:56
2019-11-08 19:41:19
2019-11-08 19:37:04
2019-11-08 19:36:12
2019-11-08 19:29:02
2019-11-08 19:28:30
2019-11-08 19:21:40
2019-11-08 19:15:48
2019-11-08 19:06:46
2019-11-08 19:04:13
2019-11-08 18:59:34
2019-11-08 18:54:46
2019-11-08 18:52:43
2019-11-08 18:50:16
2019-11-08 18:47:33
2019-11-08 18:47:06
2019-11-08 18:42:05
2019-11-08 18:37:26
2019-11-08 18:19:43
2019-11-08 18:16:11
2019-11-08 18:14:10
2019-11-08 18:09:48
2019-11-08 18:09:09
2019-11-08 18:06:38
2019-11-08 18:06:16
2019-11-08 17:56:27
2019-11-08 17:50:41
2019-11-08 17:50:21
2019-11-08 17:48:37
2019-11-08 17:46:35
2019-11-08 17:45:09
2019-11-08 17:43:39
2019-11-08 17:39:17
2019-11-08 17:37:16
2019-11-08 17:33:41
2019-11-08 17:30:26
2019-11-08 17:22:23
2019-11-08 17:21:45
2019-11-08 17:18:47
2019-11-08 17:14:12
2019-11-08 17:14:02
2019-11-08 17:12:18
2019-11-08 17:10:17
2019-11-08 17:05:59
2019-11-08 17:05:37
2019-11-08 17:01:24
2019-11-08 16:56:27


2019-11-08 03:20:45
2019-11-08 03:20:44
2019-11-08 03:20:40
2019-11-08 03:20:40
2019-11-08 03:20:40
2019-11-08 03:20:37
2019-11-08 03:20:32
2019-11-08 03:20:26
2019-11-08 03:20:23
2019-11-08 03:20:23
2019-11-08 03:20:22
2019-11-08 03:20:11
2019-11-08 03:20:10
2019-11-08 03:20:04
2019-11-08 03:20:03
2019-11-08 03:19:55
2019-11-08 03:19:37
2019-11-08 03:19:31
2019-11-08 03:19:25
2019-11-08 03:19:18
2019-11-08 03:19:07
2019-11-08 03:19:04
2019-11-08 03:18:58
2019-11-08 03:18:54
2019-11-08 03:18:35
2019-11-08 03:18:23
2019-11-08 03:18:20
2019-11-08 03:18:07
2019-11-08 03:18:04
2019-11-08 03:17:59
2019-11-08 03:17:52
2019-11-08 03:17:52
2019-11-08 03:17:48
2019-11-08 03:17:47
2019-11-08 03:17:45
2019-11-08 03:17:42
2019-11-08 03:17:19
2019-11-08 03:17:18
2019-11-08 03:17:14
2019-11-08 03:17:11
2019-11-08 03:17:01
2019-11-08 03:16:51
2019-11-08 03:16:51
2019-11-08 03:16:43
2019-11-08 03:16:37
2019-11-08 03:16:33
2019-11-08 03:16:31
2019-11-08 03:16:30
2019-11-08 03:16:29
2019-11-08 03:16:25


2019-11-05 18:48:44
2019-11-05 18:46:21
2019-11-05 18:43:54
2019-11-05 18:40:20
2019-11-05 18:39:23
2019-11-05 18:38:19
2019-11-05 18:34:40
2019-11-05 18:31:40
2019-11-05 18:29:47
2019-11-05 18:20:21
2019-11-05 18:20:06
2019-11-05 18:19:34
2019-11-05 18:16:44
2019-11-05 18:08:02
2019-11-05 18:03:10
2019-11-05 17:54:35
2019-11-05 17:53:16
2019-11-05 17:52:38
2019-11-05 17:46:50
2019-11-05 17:45:11
2019-11-05 17:45:10
2019-11-05 17:40:57
2019-11-05 17:40:12
2019-11-05 17:33:23
2019-11-05 17:31:27
2019-11-05 17:15:21
2019-11-05 17:14:43
2019-11-05 17:12:56
2019-11-05 17:12:45
2019-11-05 17:08:00
2019-11-05 17:01:30
2019-11-05 16:55:47
2019-11-05 16:53:37
2019-11-05 16:49:59
2019-11-05 16:49:59
2019-11-05 16:47:36
2019-11-05 16:42:44
2019-11-05 16:40:53
2019-11-05 16:39:19
2019-11-05 16:38:49
2019-11-05 16:38:04
2019-11-05 16:31:24
2019-11-05 16:30:59
2019-11-05 16:23:31
2019-11-05 16:23:21
2019-11-05 16:22:42
2019-11-05 16:20:21
2019-11-05 16:19:47
2019-11-05 16:17:02
2019-11-05 16:16:46


2019-11-04 20:06:38
2019-11-04 20:05:00
2019-11-04 20:01:43
2019-11-04 20:00:27
2019-11-04 19:53:30
2019-11-04 19:50:17
2019-11-04 19:50:09
2019-11-04 19:48:07
2019-11-04 19:47:17
2019-11-04 19:46:56
2019-11-04 19:45:16
2019-11-04 19:44:58
2019-11-04 19:44:28
2019-11-04 19:44:15
2019-11-04 19:37:38
2019-11-04 19:36:39
2019-11-04 19:36:16
2019-11-04 19:35:50
2019-11-04 19:35:47
2019-11-04 19:35:14
2019-11-04 19:34:51
2019-11-04 19:34:47
2019-11-04 19:33:43
2019-11-04 19:33:28
2019-11-04 19:25:41
2019-11-04 19:25:25
2019-11-04 19:16:48
2019-11-04 19:16:06
2019-11-04 19:14:28
2019-11-04 19:07:08
2019-11-04 19:06:34
2019-11-04 19:05:06
2019-11-04 19:03:06
2019-11-04 18:57:27
2019-11-04 18:53:04
2019-11-04 18:51:31
2019-11-04 18:41:23
2019-11-04 18:34:50
2019-11-04 18:25:59
2019-11-04 18:22:30
2019-11-04 18:17:56
2019-11-04 18:08:43
2019-11-04 18:03:34
2019-11-04 18:03:05
2019-11-04 17:55:33
2019-11-04 17:36:39
2019-11-04 17:20:05
2019-11-04 17:19:01
2019-11-04 17:09:31
2019-11-04 17:03:47


2019-11-01 16:56:41
2019-11-01 16:46:18
2019-11-01 16:39:25
2019-11-01 16:25:09
2019-11-01 16:05:51
2019-11-01 15:43:49
2019-11-01 15:30:17
2019-11-01 15:26:15
2019-11-01 15:21:27
2019-11-01 15:20:21
2019-11-01 15:08:01
2019-11-01 15:06:26
2019-11-01 15:03:04
2019-11-01 15:03:00
2019-11-01 14:52:49
2019-11-01 14:51:20
2019-11-01 14:47:06
2019-11-01 14:43:38
2019-11-01 14:43:23
2019-11-01 14:37:55
2019-11-01 13:38:28
2019-11-01 13:35:35
2019-11-01 13:31:31
2019-11-01 13:30:00
2019-11-01 13:25:55
2019-11-01 13:18:10
2019-11-01 13:05:52
2019-11-01 12:47:08
2019-11-01 12:27:31
2019-11-01 11:57:14
2019-11-01 11:36:09
2019-11-01 10:42:12
2019-11-01 09:36:08
2019-11-01 09:12:38
2019-11-01 09:05:42
2019-11-01 07:59:23
2019-11-01 07:15:13
2019-11-01 05:57:48
2019-11-01 05:45:55
2019-11-01 05:31:34
2019-11-01 05:12:12
2019-11-01 04:49:51
2019-11-01 04:23:37
2019-11-01 04:11:17
2019-11-01 04:08:06
2019-11-01 03:34:47
2019-11-01 03:15:44
2019-11-01 03:10:20
2019-11-01 02:49:03
2019-11-01 02:38:58


2019-10-31 02:14:59
2019-10-31 02:12:19
2019-10-31 02:11:40
2019-10-31 02:08:13
2019-10-31 02:01:08
2019-10-31 01:58:04
2019-10-31 01:55:39
2019-10-31 01:53:45
2019-10-31 01:52:26
2019-10-31 01:50:38
2019-10-31 01:49:51
2019-10-31 01:48:58
2019-10-31 01:45:39
2019-10-31 01:45:14
2019-10-31 01:41:18
2019-10-31 01:41:06
2019-10-31 01:39:08
2019-10-31 01:37:29
2019-10-31 01:28:44
2019-10-31 01:27:14
2019-10-31 01:24:59
2019-10-31 01:24:09
2019-10-31 01:22:01
2019-10-31 01:21:54
2019-10-31 01:20:42
2019-10-31 01:20:28
2019-10-31 01:19:21
2019-10-31 01:18:31
2019-10-31 01:17:05
2019-10-31 01:16:03
2019-10-31 01:15:21
2019-10-31 01:13:43
2019-10-31 01:12:47
2019-10-31 01:11:52
2019-10-31 01:11:41
2019-10-31 01:11:14
2019-10-31 01:11:05
2019-10-31 01:10:41
2019-10-31 01:09:24
2019-10-31 01:07:17
2019-10-31 01:06:22
2019-10-31 01:03:21
2019-10-31 01:02:54
2019-10-31 01:02:48
2019-10-31 01:01:39
2019-10-31 01:01:23
2019-10-31 01:00:50
2019-10-31 00:59:19
2019-10-31 00:59:08
2019-10-31 00:58:34


In [21]:
data_df = pd.read_json("saved_tweets.json", orient='records')
data_df.head(20)

""


In [ ]:
connection.close()